In [15]:
import re
import csv
from time import sleep
from bs4 import BeautifulSoup
import requests

headers = {
    'accept': '*/*',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.9',
    'referer': 'https://www.google.com',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36 Edg/85.0.564.44'
}

def get_article(card):
    """Extract article information from the raw html"""
    headline = card.find('h4', 's-title').text
    source = card.find("span", 's-source').text
    posted = card.find('span', 's-time').text.replace('·', '').strip()
    description = card.find('p', 's-desc').text.strip()
    raw_link = card.find('a').get('href')
    unquoted_link = requests.utils.unquote(raw_link)
    pattern = re.compile(r'RU=(.+)\/RK')
    clean_link = re.search(pattern, unquoted_link).group(1)
    
    article = (headline, source, posted, description, clean_link)
    return article

def get_the_news(search):
    """Run the main program"""
    template = 'https://news.search.yahoo.com/search?p={}'
    url = template.format(search)
    articles = []
    links = set()
    
    while True:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        cards = soup.find_all('div', 'NewsArticle')
        
        # extract articles from page
        for card in cards:
            article = get_article(card)
            link = article[-1]
            if not link in links:
                links.add(link)
                articles.append(article)        
                
        # find the next page
        try:
            url = soup.find('a', 'next').get('href')
            sleep(1)
        except AttributeError:
            break
            
    # save article data
    with open('results.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Headline', 'Source', 'Posted', 'Description', 'Link'])
        writer.writerows(articles)
        
    return articles

In [12]:
# run the main program
articles = get_the_news('cryptocurrency')

In [13]:
# show the first 4 records
articles[0:4]

[('First Mover: Ether Steals Limelight With New All-Time High Price as Bitcoiners HODL',
  'CoinDesk via Yahoo Finance',
  '1 hour ago',
  '“This period of consolidation is building a solid base, giving those who wish to sell bitcoin plenty...',
  'https://finance.yahoo.com/news/first-mover-ether-steals-limelight-140758906.html'),
 ('Bitcoin’s Turbulence Helps Kindle Rally in Largest Rival Ether',
  'Bloomberg via Yahoo Finance',
  '2 hours ago',
  'The world’s largest cryptocurrency hovered around $37,000 on Tuesday after major gyrations earlier...',
  'https://finance.yahoo.com/news/bitcoin-turbulence-helps-kindle-rally-071510093.html'),
 ('Thieves Grab $451K in Cash From Hong Kong Crypto Trader',
  'CoinDesk via Yahoo Finance',
  '1 hour ago',
  'Hong Kong police are searching for robbers who lured a female cryptocurrency trader into an office...',
  'https://finance.yahoo.com/news/thieves-grab-451k-cash-hong-140008895.html'),
 ('Bitcoin Steady as Analysts Say Getting Back to $40,00